# HATLAS-SGP master catalogue: Flags

In [1]:
import numpy as np
from astropy.table import Table, Column
import itertools

import glob

from herschelhelp_internal.flagging import flag_outliers

In [2]:

FIELD = 'SGP'
SUFFIX = '20180221'
# On XMM-LSS we have a list of tiles due to the large size of the field. 
# Therefore, we here process each in turn
file_list = glob.glob('../../dmu1/dmu1_ml_SGP/data/tiles/sub_catalogue_sgp_{}_*.fits'.format(SUFFIX))

In [3]:
g_bands = ["OmegaCAM g", "DECam g", "GPC1 g"]
r_bands = ["OmegaCAM r", "DECam r", "GPC1 r"]
i_bands = ["OmegaCAM i", "DECam i", "GPC1 i"]
z_bands = ["OmegaCAM z", "DECam z", "GPC1 z", "VISTA Z"]
y_bands = [              "DECam y", "GPC1 y", "VISTA Y"]

all_bands = [g_bands, r_bands, i_bands, z_bands, y_bands]

## 1. Magnitudes and magnitude erros

In [4]:
def flag_mag(master_catalogue, flagcol, mask):
    
    # Add flag columns if does not exist
    if flagcol not in master_catalogue.colnames:
        master_catalogue.add_column(Column(data=np.zeros(len(master_catalogue), dtype=bool), name=flagcol))
    
    # Flagged
    master_catalogue[flagcol][mask] = np.ones(len(mask), dtype=bool)
    print('    Number of flagged objects:', len(master_catalogue[flagcol][mask]))
    return master_catalogue

### 1.a Pan-STARRS Aperture and Total magnitude errors

In [5]:
def panstars_loopable(master_catalogue):
    ## dmu0: Pan-STARRS stack cat 
    gpc1_err = 0.0010860000038519502
    bands = ["GPC1 g", "GPC1 r", "GPC1 i", "GPC1 z", "GPC1 y"]

    for i, band in enumerate(bands):
        print(band)
        basecol = band.replace(" ", "_").lower()
    
        ecol_ap, ecol_tot = "merr_ap_{}".format(basecol), "merr_{}".format(basecol)
        flagcol_ap, flagcol_tot = "flag_ap_{}".format(basecol), "flag_{}".format(basecol)
    
        mask_ap  = np.where(master_catalogue[ecol_ap]  == gpc1_err)[0]
        mask_tot = np.where(master_catalogue[ecol_tot] == gpc1_err)[0]
    
        print('  Aperture magnitude')
        master_catalogue = flag_mag(master_catalogue, flagcol_ap, mask_ap)
        print('  Total magnitude')
        master_catalogue = flag_mag(master_catalogue, flagcol_tot, mask_tot)
        
    return master_catalogue
    

## 2. Outliers

In [6]:
def outliers_loopable(master_catalogue):
    for band_of_a_kind in all_bands:
        for band1, band2 in itertools.combinations(band_of_a_kind, 2):
            #print(band1, band2)
    
            basecol1, basecol2 = band1.replace(" ", "_").lower(), band2.replace(" ", "_").lower()
            
            # Aperture mag
            col1, col2 = "m_ap_{}".format(basecol1), "m_ap_{}".format(basecol2)
            ecol1, ecol2 = "merr_ap_{}".format(basecol1), "merr_ap_{}".format(basecol2)
            flagcol1, flagcol2 = "flag_ap_{}".format(basecol1), "flag_ap_{}".format(basecol2)
        
            try:
                master_catalogue = flag_outliers(master_catalogue, col1, col2,
                          ecol1, ecol2,
                          flagcol1, flagcol2,
                          labels=("{} (aperture)".format(band1), "{} (aperture)".format(band2)))
            except KeyError:
                print("One of {} and {} is not present in the catalogue.".format(col1, col2))
                      
        
            # Tot mag
            col1, col2 = "m_{}".format(basecol1), "m_{}".format(basecol2)              
            ecol1, ecol2 = "merr_{}".format(basecol1), "merr_{}".format(basecol2)              
            flagcol1, flagcol2 = "flag_{}".format(basecol1), "flag_{}".format(basecol2)
              
            try:
                master_catalogue = flag_outliers(master_catalogue, col1, col2, 
                          ecol1, ecol2,
                          flagcol1, flagcol2,
                          labels=("{} (total)".format(band1), "{} (total)".format(band2)))   
            except KeyError:
                print("One of {} and {} is not present in the catalogue.".format(col1, col2))  
    return master_catalogue             
                      

## 3. Save table

In [7]:
for tile in file_list:
    master_catalogue = Table.read(tile)
    master_catalogue = panstars_loopable(master_catalogue)
   # master_catalogue = irac_loopable(master_catalogue)
    master_catalogue = outliers_loopable(master_catalogue)
    
    #Merge any aperture flags
    for col in master_catalogue.colnames:
        if col.startswith("flag_ap_"):
            try:
                master_catalogue[col.replace("_ap_", "_")] = (master_catalogue[col.replace("_ap_", "_")] |
                                                          master_catalogue[col])
                master_catalogue.remove_column(col)
            except KeyError:
                print("{} only has aperture flags.".format(col))
                master_catalogue.rename_column(col, col.replace("_ap_", "_"))
    
    
    
    file_num = tile.split('_')[6].split('.')[0]
    
    flag_cols = ["help_id"]
    for col in master_catalogue.colnames:
        if col.startswith("flag_"):
            flag_cols += [col]
    new_catname = "./data/tiles/{}_{}_{}_flags.fits".format(FIELD.lower(),SUFFIX, file_num)
    master_catalogue[flag_cols].write(new_catname, overwrite = True)

GPC1 g
  Aperture magnitude
    Number of flagged objects: 0
  Total magnitude
    Number of flagged objects: 0
GPC1 r
  Aperture magnitude
    Number of flagged objects: 0
  Total magnitude
    Number of flagged objects: 0
GPC1 i
  Aperture magnitude
    Number of flagged objects: 0
  Total magnitude
    Number of flagged objects: 0
GPC1 z
  Aperture magnitude
    Number of flagged objects: 0
  Total magnitude
    Number of flagged objects: 0
GPC1 y
  Aperture magnitude
    Number of flagged objects: 0
  Total magnitude
    Number of flagged objects: 0
No sources have both OmegaCAM g (aperture) and DECam g (aperture) values.

No sources have both OmegaCAM g (total) and DECam g (total) values.

No sources have both OmegaCAM g (aperture) and GPC1 g (aperture) values.

No sources have both OmegaCAM g (total) and GPC1 g (total) values.

No sources have both DECam g (aperture) and GPC1 g (aperture) values.

No sources have both DECam g (total) and GPC1 g (total) values.

No sources have bo